### Download Flowers dataset

In [0]:
#You can download the data manually as well instead of using 'wget'
!wget http://download.tensorflow.org/example_images/flower_photos.tgz --quiet

In [2]:
#Check if file is downloaded
!ls -l

total 223456
-rw-r--r-- 1 root root 228813984 Feb 10  2016 flower_photos.tgz
drwxr-xr-x 1 root root      4096 Jan 13 16:38 sample_data


In [3]:
#Unzip the data
!tar -xvf flower_photos.tgz

flower_photos/
flower_photos/roses/
flower_photos/roses/14810868100_87eb739f26_m.jpg
flower_photos/roses/1446090416_f0cad5fde4.jpg
flower_photos/roses/15319767030_e6c5602a77_m.jpg
flower_photos/roses/15032112248_30c5284e54_n.jpg
flower_photos/roses/7211616670_2d49ecb3a5_m.jpg
flower_photos/roses/15674450867_0ced942941_n.jpg
flower_photos/roses/17158274118_00ec99a23c.jpg
flower_photos/roses/14019883858_e5d2a0ec10_n.jpg
flower_photos/roses/8035908422_87220425d2_n.jpg
flower_photos/roses/14747962886_2bff6bb323_m.jpg
flower_photos/roses/4356781875_92c5cd93c0.jpg
flower_photos/roses/8524505546_b242bd4928_n.jpg
flower_photos/roses/9406573080_60eab9278e_n.jpg
flower_photos/roses/6039330368_c30ed224c4_m.jpg
flower_photos/roses/14414100710_753a36fce9.jpg
flower_photos/roses/3292434691_392071d702_n.jpg
flower_photos/roses/5273722065_c85d8543c2_m.jpg
flower_photos/roses/2215318403_06eb99176a.jpg
flower_photos/roses/172311368_49412f881b.jpg
flower_photos/roses/15255964454_0a64eb67fa.jpg
flower_pho

In [4]:
#Check how data is organized
!ls -l flower_photos

total 616
drwx------ 2 270850 5000  36864 Feb 10  2016 daisy
drwx------ 2 270850 5000  53248 Feb 10  2016 dandelion
-rw-r----- 1 270850 5000 418049 Feb  9  2016 LICENSE.txt
drwx------ 2 270850 5000  36864 Feb 10  2016 roses
drwx------ 2 270850 5000  40960 Feb 10  2016 sunflowers
drwx------ 2 270850 5000  40960 Feb 10  2016 tulips


### Build batch generator

In [5]:
import tensorflow as tf

In [6]:
tf.test.gpu_device_name()

''

In [0]:
#Define some parameters
img_size = 60
img_depth = 3

Create an ImageDataGenerator object, it can also split data between train and test.

In [0]:
#ImageDataGenerator declaration with 20% data as test (80% for training)
img_generator= tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

ImageDataGenerator can read images directory and also resize them if needed

In [9]:
#Build training generator. 
train_generator = img_generator.flow_from_directory('flower_photos',
                                                    target_size=(img_size, img_size),
                                                    subset='training',
                                                    batch_size=64)

#Build test generator
test_generator = img_generator.flow_from_directory('flower_photos',
                                                   target_size=(img_size, img_size),                                                   
                                                   subset='validation',
                                                   batch_size=64)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


ImageDataGenerator returns 64 images and their labels

In [10]:
#Lets check the features (images) and Labels (flower class) returned by ImageDataGenerator
X, y = next(train_generator)

print('Input features shape', X.shape)
print('Actual labels shape', y.shape)

Input features shape (64, 60, 60, 3)
Actual labels shape (64, 5)


### Build CNN Model

In [11]:
#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()

#normalize data
model.add(tf.keras.layers.BatchNormalization(input_shape=(img_size,img_size,3,)))

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization(input_shape=(img_size,img_size,3,)))

#Add Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

#normalize data
model.add(tf.keras.layers.BatchNormalization(input_shape=(img_size,img_size,3,)))

#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Add Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))

#Add Dropout
model.add(tf.keras.layers.Dropout(0.25))

#Add Output Layer
model.add(tf.keras.layers.Dense(5, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
#Model Summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 60, 60, 3)         12        
_________________________________________________________________
conv2d (Conv2D)              (None, 58, 58, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 58, 58, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 56, 56, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 28, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0

### Train the model

In [0]:
model.fit_generator(train_generator, 
                          epochs=200,
                          steps_per_epoch= 2939//64,  #Number of training images//batch_size
                          validation_data=test_generator,
                          validation_steps = 731//64) #Number of test images//batch_size

Epoch 1/200
45/45 [==============================] - 13s 281ms/step - loss: 3.4326 - acc: 0.3221 - val_loss: 1.7119 - val_acc: 0.2599
Epoch 2/200
45/45 [==============================] - 12s 273ms/step - loss: 1.5384 - acc: 0.3788 - val_loss: 7.3582 - val_acc: 0.2628
Epoch 3/200
45/45 [==============================] - 12s 270ms/step - loss: 1.3816 - acc: 0.4143 - val_loss: 15.8429 - val_acc: 0.2940
Epoch 4/200
45/45 [==============================] - 12s 266ms/step - loss: 1.3439 - acc: 0.4219 - val_loss: 12.7242 - val_acc: 0.3310
Epoch 5/200
45/45 [==============================] - 12s 272ms/step - loss: 1.3147 - acc: 0.4424 - val_loss: 11.3076 - val_acc: 0.3949
Epoch 6/200
45/45 [==============================] - 12s 265ms/step - loss: 1.2672 - acc: 0.4661 - val_loss: 7.8129 - val_acc: 0.4545
Epoch 7/200
45/45 [==============================] - 12s 262ms/step - loss: 1.2324 - acc: 0.4936 - val_loss: 4.7577 - val_acc: 0.4957
Epoch 8/200
45/45 [==============================] - 12s 26

In [13]:
#Time taken on CPU
model.fit_generator(train_generator, 
                          epochs=1,
                          steps_per_epoch= 2939//64,  #Number of training images//batch_size
                          validation_data=test_generator,
                          validation_steps = 731//64) #Number of test images//batch_size

45/45 [==============================] - 65s 1s/step - loss: 7.3793 - acc: 0.3437 - val_loss: 1.9699 - val_acc: 0.2670
